In [60]:
import RDatasets
import Impute

┌ Info: Precompiling Impute [f7bf1975-0170-51b9-8c5f-a992d46b9575]
└ @ Base loading.jl:1317
┌ Warning: Module HTTP with build ID 158906228973362 is missing from the cache.
│ This may mean HTTP [cd3eb016-35fb-5094-929b-558a96fad6f3] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1008
┌ Warning: Module HTTP with build ID 158906228973362 is missing from the cache.
│ This may mean HTTP [cd3eb016-35fb-5094-929b-558a96fad6f3] does not support precompilation but is imported by a module that does.
└ @ Base loading.jl:1008
┌ Info: Skipping precompilation since __precompile__(false). Importing Impute [f7bf1975-0170-51b9-8c5f-a992d46b9575].
└ @ Base loading.jl:1025
┌ Info: Precompiling DataDeps [124859b0-ceae-595e-8997-d05f6a7a8dfe]
└ @ Base loading.jl:1317
┌ Warning: Module HTTP with build ID 158906228973362 is missing from the cache.
│ This may mean HTTP [cd3eb016-35fb-5094-929b-558a96fad6f3] does not support precompilation but is imported by a module

In [12]:
using Plots
using DataFrames
import CSV
using MLJ

In [40]:
features = DataFrame(CSV.File("features.csv",missingstring="missing"));
train = DataFrame(CSV.File("train.csv"));
test = DataFrame(CSV.File("test.csv"));
stores = DataFrame(CSV.File("stores.csv"));
features.StoreType = stores[features.Store,:Type];
features.Size = stores[features.Store,:Size];
features.Id1 = string.(features.Store,"_",features.Date);
select!(features, Not([:Store,:Date]));

In [41]:
# train.Id = string.(train.Store,"_",train.Dept,"_",train.Date);
train.Id1 = string.(train.Store,"_",train.Date);
# select!(train, Not([:Store,:Date,:IsHoliday]));
# select!(train, :Id,:Weekly_Sales,:IsHoliday);

# test.Id = string.(test.Store,"_",test.Dept,"_",test.Date);
test.Id1 = string.(test.Store,"_",test.Date);
# select!(test, Not([:Store,:Date,:IsHoliday]));
# select!(test, :Id,:IsHoliday);

In [42]:
train = innerjoin(features,train,on=:Id1,makeunique=true)
train.Id = string.(train.Store,"_",train.Dept,"_",train.Date);
# select!(train, Not([:Store,:Date,:IsHoliday,:Id1]));
# target = select(train, :Weekly_Sales);
select!(train, Not([:Id1,:IsHoliday_1]));
train;

In [43]:
test = innerjoin(features,test,on=:Id1,makeunique=true)
test.Id = string.(test.Store,"_",test.Dept,"_",test.Date);
# select!(test, Not([:Store,:Date,:IsHoliday,:Id1]));
select!(test, Not([:Id1,:IsHoliday_1]));
test;

In [44]:
train = coerce(train, :StoreType => Multiclass);

In [45]:
schema(train)

┌──────────────┬──────────────────────────────────┬─────────────────────────────
│ _.names      │ _.types                          │ _.scitypes                 ⋯
├──────────────┼──────────────────────────────────┼─────────────────────────────
│ Temperature  │ Float64                          │ Continuous                 ⋯
│ Fuel_Price   │ Float64                          │ Continuous                 ⋯
│ MarkDown1    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown2    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown3    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown4    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ MarkDown5    │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ CPI          │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ Unemployment │ Union{Missing, Float64}          │ Union{Missing, Continuous} ⋯
│ IsHoliday    │ Bool       

In [65]:
# y, X = unpack(train, ==(:Weekly_Sales), colname -> true)
y, X = unpack(train, 
    ==(:Weekly_Sales), 
    colname -> colname != :Date &&  colname !=:Id
#     !=([:Date, :Id])
#     !=(:Date)
    );

In [72]:
schema(X)

┌──────────────┬──────────────────────────────────┬───────────────┐
│ _.names      │ _.types                          │ _.scitypes    │
├──────────────┼──────────────────────────────────┼───────────────┤
│ Temperature  │ Float64                          │ Continuous    │
│ Fuel_Price   │ Float64                          │ Continuous    │
│ MarkDown1    │ Float64                          │ Continuous    │
│ MarkDown2    │ Float64                          │ Continuous    │
│ MarkDown3    │ Float64                          │ Continuous    │
│ MarkDown4    │ Float64                          │ Continuous    │
│ MarkDown5    │ Float64                          │ Continuous    │
│ CPI          │ Float64                          │ Continuous    │
│ Unemployment │ Float64                          │ Continuous    │
│ IsHoliday    │ Bool                             │ Count         │
│ StoreType    │ CategoricalValue{String, UInt32} │ Multiclass{3} │
│ Size         │ Int64                          

In [71]:
X = Impute.filter(X; dims=:rows)
# X = Impute.substitute(X)

,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5
,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,59.11,3.297,10382.9,6115.67,215.07,2406.62,6551.42
2,62.25,3.308,6074.12,254.39,51.98,427.39,5988.57
3,60.14,3.236,410.31,98.0,55805.5,8.0,554.92
4,48.91,3.172,5629.51,68.0,1398.11,2084.64,20475.3
5,43.93,3.158,4640.65,19.0,105.02,3639.42,14461.8
6,51.63,3.159,5011.32,67.0,347.37,225.79,4011.37
7,47.96,3.112,2725.36,40.48,634.7,24.9,2739.43
8,44.55,3.129,5762.1,46011.4,260.36,983.65,4735.78
9,49.01,3.157,6277.39,21813.2,143.1,1450.13,8483.0


In [82]:
models(matching(X, y))[3]

Constant regressor (Deterministic).
→ based on [MLJModels](https://github.com/alan-turing-institute/MLJModels.jl).
→ do `@load DeterministicConstantRegressor pkg="MLJModels"` to use the model.
→ do `?DeterministicConstantRegressor` for documentation.
(name = "DeterministicConstantRegressor",
 package_name = "MLJModels",
 is_supervised = true,
 docstring = "Constant regressor (Deterministic).\n→ based on [MLJModels](https://github.com/alan-turing-institute/MLJModels.jl).\n→ do `@load DeterministicConstantRegressor pkg=\"MLJModels\"` to use the model.\n→ do `?DeterministicConstantRegressor` for documentation.",
 hyperparameter_ranges = (),
 hyperparameter_types = (),
 hyperparameters = (),
 implemented_methods = [:fit, :predict],
 is_pure_julia = true,
 is_wrapper = false,
 iteration_parameter = nothing,
 load_path = "MLJModels.DeterministicConstantRegressor",
 package_license = "MIT",
 package_url = "https://github.com/alan-turing-institute/MLJModels.jl",
 package_uuid = "d491faf4-2d78-

In [87]:
@load DeterministicConstantRegressor pkg="MLJModels"

import MLJModels ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/alex/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


MLJModels.DeterministicConstantRegressor

In [90]:
knc = @load KNeighborsClassifier

import MLJScikitLearnInterface

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/alex/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


LoadError: ArgumentError: Package MLJScikitLearnInterface not found in current path:
- Run `import Pkg; Pkg.add("MLJScikitLearnInterface")` to install the MLJScikitLearnInterface package.
